### Train RLHF using the data from database

Let's take a look of the QA data and process it for training.

In [ ]:
QA_CSV_HEADER_ID = 'ID'
QA_CSV_HEADER_QUESTION = 'Question'
QA_CSV_HEADER_ANSWER = 'Answer'
QA_CSV_HEADER_VOTE_STATUS = 'Vote Status'
QA_CSV_HEADER_TIMESTAMPS = 'Timestamp'
QA_CSV_HEADER = (
    QA_CSV_HEADER_ID,
    QA_CSV_HEADER_QUESTION,
    QA_CSV_HEADER_ANSWER,
    QA_CSV_HEADER_VOTE_STATUS,
    QA_CSV_HEADER_TIMESTAMPS
)

In [ ]:
my_data_pd = qa_database.retrieve_all_question_answers_as_pandas()
my_data_pd.head()

,ID,Question,Answer,Vote Status,Timestamp
0,1,who founded YC and what does YC do,Y Combinator is a startup accelerator that pro...,up,2023-07-24 08:28:29.461045
1,2,what are the top YC companies,I am looking for a list of the top YC companie...,down,2023-07-24 08:29:17.521467
2,3,How much does top 10 YC companies worth,"According to the latest Crunchbase data, the 1...",down,2023-07-24 08:29:58.248998
3,4,What's the meaning of life,"The meaning of life is to be happy, to be usef...",down,2023-07-24 19:26:04.713297
4,5,What is the meaning of life,The meaning of life is to find your gift. The ...,up,2023-07-24 19:26:28.676427


In [ ]:
my_data_pd = my_data_pd[my_data_pd[QA_CSV_HEADER_VOTE_STATUS]=="up"]
my_data_pd.shape

(3, 5)

In [ ]:
from datasets import Dataset

my_data_pd = my_data_pd[[QA_CSV_HEADER_ID,
                        QA_CSV_HEADER_QUESTION,
                        QA_CSV_HEADER_ANSWER]]
print("My local database has {} samples".format(my_data_pd.shape[0]))
dataset = Dataset.from_dict(my_data_pd)
dataset

My local database has 3 samples


Dataset({
    features: ['ID', 'Question', 'Answer'],
    num_rows: 3
})

### Train with RLHF

In [ ]:
# from pykoi.rlhf.rlhf import RLHFConfig, SFT ##, RewardTrainer, RL

config = pykoi.RLHFConfig(base_model_path="tiiuae/falcon-7b", # "meta-llama/Llama-2-7b-hf", 
                          dataset_type="local_db", # ""
                          dataset_name="/home/ubuntu/git/pykoi/test_data/train-00000-of-00020.parquet"
                          )

rlhf_step1_sft = pykoi.SFT(config)

/opt/conda/envs/724b/lib/python3.10/site-packages/trl/trainer/utils.py:246: UserWarning: The passed formatting_func has more than one argument. Usually that function should have a single argument `example` which corresponds to the dictonnary returned by each element of the dataset. Make sure you know what you are doing.
  warnings.warn(


My local database has 3 samples
Size of the train set: 2.               Size of the validation set: 1


Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.18s/it]
/opt/conda/envs/724b/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


ValueError: Please specify `target_modules` in `peft_config`

t=2023-07-24T23:17:11+0000 lvl=warn msg="failed to open private leg" id=2d43a7ad08a5 privaddr=localhost:59175 err="dial tcp 127.0.0.1:59175: connect: connection refused"
t=2023-07-24T23:20:16+0000 lvl=warn msg="failed to open private leg" id=c68d1d85cbb4 privaddr=localhost:59175 err="dial tcp 127.0.0.1:59175: connect: connection refused"
t=2023-07-24T23:22:21+0000 lvl=warn msg="failed to open private leg" id=ea8c21fff5bd privaddr=localhost:59175 err="dial tcp 127.0.0.1:59175: connect: connection refused"
t=2023-07-24T23:23:42+0000 lvl=warn msg="failed to open private leg" id=1ec689ec7aa4 privaddr=localhost:59175 err="dial tcp 127.0.0.1:59175: connect: connection refused"


In [ ]:
rlhf_step1_sft.train_and_save("../../rlhf_tests/step1_07211538")

NameError: name 'rlhf_step1_sft' is not defined

### Insert my data to the DB

In [ ]:
import pandas as pd
my_stackoverflow_dataset = pd.read_csv("/home/ubuntu/pykoi/pykoi/my_sql_data.csv", index_col=0)
my_stackoverflow_dataset.head(100)

,ID,Question,Answer,Vote Status,Timestamp
0,1,Who is Rachel?,CEO of CambioML,up,2023-07-20 23:19:25.202246
1,2,Who is Jared?,CTO of CambioML,up,2023-07-20 23:19:25.206972
2,3,Who is Kimi?,Nobody of CambioML,down,2023-07-20 23:19:25.210210
0,12891264,I am using jQuery fileupload plugin and I want...,"Looking at the library code, seems all events ...",NaN,NaN
1,588860,When reading about the Big Bang you’ll commonl...,No. The local energy *density* is reduced as t...,NaN,NaN
...,...,...,...,...,...
92,16617081,here is my object with sample data\n\n```\nObj...,"Firstly, if you've just got a date, I would su...",NaN,NaN
93,7690401,"Trying to add ""import"" statement to my new sca...",Make sure that you have a Java SDK configured ...,NaN,NaN
94,42071634,Is there any way to define a static method in ...,The simplest solution is probably to have the ...,NaN,NaN
95,42071634,Is there any way to define a static method in ...,The simplest solution is probably to have the ...,NaN,NaN


In [ ]:
for row in my_stackoverflow_dataset.iloc[3:100].to_dict('records'):
    qa_id = qa_database.insert_question_answer(question=row[QA_CSV_HEADER_QUESTION],
                                       answer=row[QA_CSV_HEADER_ANSWER])
    qa_database.update_vote_status(id=qa_id, vote_status="up") #row[QA_CSV_HEADER_VOTE_STATUS])

In [ ]:
qa_database

In [ ]:
# dataset = Dataset.from_dict(my_data_pd)